# Old code and ideas

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import requests
import json
import csv
import time
import datetime as dt
import math
import itertools

In [2]:
# first subreddit
subreddit = 'mentalhealth'

url = 'https://api.pushshift.io/reddit/search/submission?subreddit=mentalhealth'
headers = {'User-agent': 'enr11'}
res = requests.get(url, headers=headers)
res.status_code

if res.status_code == 200:
    subreddit = res.json()

subreddit.keys()

subreddit['data'][0]

posts = subreddit['data']
topics_dict = { 'title':[],
               'selftext':[],
                'id':[],
                'subreddit': []
              }

for i in range(2):
    for post in posts:
        topics_dict['title'].append(post['title'])
        topics_dict['id'].append(post['id'])
        topics_dict['selftext'].append(post['selftext'])
        topics_dict['subreddit'].append(post['subreddit'])

topics_data = pd.DataFrame(topics_dict)
topics_data.head()

topics_data.shape

(50, 4)

In [3]:
# was running into issues pulling more than 1200, it said error: selftext. presumably because
# there were no selftext values in the pull of 50 posts after 1200
# I tried doing:
#   try:
#             posts_data['created_utc'].append(post['created_utc'])
#             posts_data['url'].append(post['full_link'])
#             posts_data['id'].append(post['id'])
#             posts_data['num_comments'].append(post['num_comments'])
#             posts_data['title'].append(post['title'])
#             posts_data['selftext'].append(post['selftext'])
#             posts_data['subreddit'].append(post['subreddit'])
#         except KeyError:
#              'selftext' = "NaN"
# but it said arrays must be the same length

In [4]:
# Subreddit to be scraped
subreddit = 'mentalhealth'

In [6]:
# h/t stack overflow 
# Set up dict for info to collect
posts_data = {'created_utc':[],
              'url':[],
              'id':[],
              'num_comments':[],
              'title':[],
              'selftext':[],
              'subreddit':[]
              }

headers = {'User-agent': 'Reddit Post Collector'}

# Set up function to return submission data
def get_submissions(**kwargs):
    res = requests.get("https://api.pushshift.io/reddit/submission/search/",
                       params=kwargs,
                       headers=headers)
    if res.status_code == 200:
        data = res.json()
        return data['data']
    else:
        print(res.status_code)

before = None
count = 0

# Collect up to 1,250 posts as long as there are posts to collect
while True and len(set(posts_data['created_utc'])) < 1250:
    print(count)
    count += 1*50
    
    posts = get_submissions(subreddit=subreddit,
                            size=50,
                            before=before,
                            sort='desc',
                            sort_type='created_utc')
    if not posts:
        break

    for post in posts:
        # Keep track of position for the next call in while loop
        before = post['created_utc']

        # Append info to posts_data dict
        posts_data['created_utc'].append(post['created_utc'])
        posts_data['url'].append(post['full_link'])
        posts_data['id'].append(post['id'])
        posts_data['num_comments'].append(post['num_comments'])
        posts_data['title'].append(post['title'])
        posts_data['selftext'].append(post['selftext'])
        posts_data['subreddit'].append(post['subreddit'])

    time.sleep(3)

# Save posts to dataframe
posts_data = pd.DataFrame(posts_data)

# Create `timestamp` column with `created_utc` translated into readable time
def get_date(created):
    return dt.datetime.fromtimestamp(created)

_timestamp = posts_data['created_utc'].apply(get_date)
posts_data = posts_data.assign(timestamp = _timestamp)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250


In [7]:
# # for after the dataframe is created and during preprocessing
# # Add lematized words to dataframe
# df = df.assign(lemmatized = tokens_lem)
# df.head(3)

# # had issues figuring out how to add lemmatized words, do it later
# # chose to do lematization because it takes in context and doesn't convert the word to a hard to interpret base

In [8]:
# properly get rid of other symbols and stopwords

In [ ]:
# count vectorizer 